In [ ]:
float(transition_probability[data['Initial State'][i])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the data from the xlsx file
data = pd.read_excel('Final_transition_mStar_2_q_0.6_plink_0.5.xlsx')

# Initialize the parameters
gamma = 0.9
epsilon = 0.001

# Initialize the state space
states = data['Initial State'].unique()

# Initialize the action space
actions = data['Action'].unique()

# Initialize the transition probability matrix
transition_probability = np.zeros((len(states), len(actions), len(states)))

# Initialize the reward matrix
reward = np.zeros((len(states), len(actions), len(states)))

# Initialize the value function
value_function = np.zeros(len(states))

# Initialize the policy
policy = np.zeros(len(states))

# Initialize the policy improvement flag
policy_stable = False

# Initialize the iteration counter
iteration = 0

# Initialize the reward matrix
for i in range(len(data)):
    transition_probability[data['Initial State'][i] - 1, data['Action'][i] - 1, data['Final State'][i] - 1] = data['Transition Probability'][i]
    reward[data['Initial State'][i] - 1, data['Action'][i] - 1, data['Final State'][i] - 1] = data['Reward'][i]

# Value iteration algorithm
while not policy_stable:
    policy_stable = True
    delta = 0
    for s in states:
        old_value = value_function[s - 1]
        action_values = np.zeros(len(actions))
        for a in actions:
            action_values[a - 1] = np.sum(transition_probability[s - 1, a - 1, :] * (reward[s - 1, a - 1, :] + gamma * value_function))
        value_function[s - 1] = np.max(action_values)
        delta = max(delta, np.abs(old_value - value_function[s - 1]))
        old_policy = policy[s - 1]
        policy[s - 1] = np.argmax(action_values) + 1
        if old_policy != policy[s - 1]:
            policy_stable = False
    iteration += 1
    if delta < epsilon:
        break

# Print the optimal policy
print('The optimal policy is:')
print(policy)

# Print the optimal value function
print('The optimal value function is:')
print(value_function)

# Print the number of iterations
print('The number of iterations is:')
print(iteration)

In [ ]:
import pandas as pd
import copy

def value_iteration(df, gamma, theta):
    # Initialize value function and policy with zeros
    n_states = df["Initial State"].nunique()
    value_function = [0] * n_states
    policy = [0] * n_states

    # Initialize utility values for each action
    action_values = {
        action: [0] * n_states for action in df["Action"].unique()
    }

    # Iterate until convergence
    while True:
        value_function_prev = copy.deepcopy(value_function)
        # Compute new value function
        for i in range(n_states):
            for action, action_value in action_values.items():
                # Compute the expected return for this action
                action_value[i] = sum(
                    df[(df["Initial State"] == i) & (df["Action"] == action)]
                    .apply(
                        lambda row: (
                            row["Transition Probability"]
                            * (row["Reward"] + gamma * value_function[row["Final State"]])
                        ),
                        axis=1,
                    )
                    .values
                )

            # Select the best action and update the value function
            best_action = max(action_values, key=lambda action: action_values[action][i])
            policy[i] = best_action
            value_function[i] = action_values[best_action][i]

        # Check for convergence
        if all(abs(value_function[i] - value_function_prev[i]) < theta for i in range(n_states)):
            break

    return policy

# Read data from Excel file
df = pd.read_excel("Final_transition_mStar_2_q_0.6_plink_0.5.xlsx")

# Run value iteration
policy = value_iteration(df, gamma=0.9, theta=1e-3)

# Print the optimal policy
print("Optimal policy:", policy)


In [ ]:
import pandas as pd
df = pd.read_excel("Final_transition_mStar_2_q_0.6_plink_0.5.xlsx")
rewards = {}
transition_probs = {}

for index, row in df.iterrows():
    initial_state = row["Initial State"]
    final_state = row["Final State"]
    action = row["Action"]
    reward = row["Reward"]
    transition_prob = row["Transition Probability"]
    # Add the reward to the rewards dictionary
    rewards[initial_state] = reward

# Add the transition probability to the transition probabilities dictionary
    if initial_state not in transition_probs:
        transition_probs[initial_state] = {}
    if action not in transition_probs[initial_state]:
        transition_probs[initial_state][action] = {}
        transition_probs[initial_state][action][final_state] = transition_prob
        
        
discount_factor = 0.9
max_error = 0.001
state_values = {state: 0 for state in rewards.keys()}
optimal_policy = {state: None for state in rewards.keys()}

while True:
    # Initialize the maximum change in state value to 0
    max_change = 0.001
    # Loop through all states
    for state in rewards.keys():
        # Initialize the value of the state to 0
        state_value = 0
    
        # Loop through all possible actions from the state
        for action in transition_probs[state].keys():
            # Initialize the expected value of the action to 0
            action_value = 0
    
            # Loop through all possible final states from the action
            for final_state in transition_probs[state][action].keys():
                # Calculate the expected value of the action
                action_value += transition_probs[state][action][final_state] * state_values[final_state]
    
            # Calculate the value of the state
            state_value = max(state_value, rewards[state] + discount_factor * action_value)
    
        # Calculate the change in state value
        change = abs(state_values[state] - state_value)
        max_change = max(max_change, change)
    
        # Update the state value and the optimal policy
        state_values[state] = state_value
        for action in transition_probs[state].keys():
            action_value = 0
            for final_state in transition_probs[state][action].keys():
                action_value += transition_probs[state][action][final_state] * state_values[final_state]
            if action_value > state_value:
                optimal_policy[state] = action
    
    # Check if the maximum change is less than the maximum error allowed
    print(max_change, max_error)
    if max_change <= max_error:
        break

print(optimal_policy)


In [ ]:
import pandas as pd
import copy as cp
data = pd.read_excel('Final_transition_mStar_3_q_0.6_plink_0.7.xlsx')

delta = 0
discount = 0.9
values = {}
optimal_policy = {}

for initial_state in data['Initial State'].unique():
    values[initial_state] = 0

while delta < 10:
    for initial_state in data['Initial State'].unique():
        max_value = cp.deepcopy(values[initial_state])
        for action in data[data['Initial State'] == initial_state]['Action Taken'].unique():
            print(initial_state,action)
            # initialize the expected reward to be 0
            expected_reward = 0
            # iterate over all final states
            for final_state in data[(data['Initial State'] == initial_state) & (data['Action Taken'] == action)]['Final State'].unique():
                # calculate the expected reward by summing over all final states
                print("Final State", final_state)
                expected_reward += data[(data['Initial State'] == initial_state) & (data['Action Taken'] == action) & (data['Final State'] == final_state)]['Expected Reward'].values[0] * data[(data['Initial State'] == initial_state) & (data['Action Taken'] == action) & (data['Final State'] == final_state)]['Transition Probability'].values[0]
                # calculate the value of taking action in the initial state
                value = expected_reward + discount * values[final_state]
                # update the max value and the optimal action if necessary
                if value > max_value:
                    max_value = cp.deepcopy(value)
                    optimal_policy[initial_state] = action
                    values[initial_state] = cp.deepcopy(max_value)
    delta = delta + 1 
    print()
                
print(optimal_policy)

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
fid_thresh = 0.87

In [5]:
data = pd.read_excel('Final_Dis2_sep_ES_fid_threshFinal_0.9both_queues_transition_mStar_3queue_length=3_q_1_plink_0.7alpha_0.01.xlsx')
    
# Initialize the parameters
gamma = 0.9
delta = 0

# Initialize the state space
states = data['Initial State'].unique()


# Initialize the action space
actions = data['Action'].unique()

V = {state:0 for state in states}
V_old = {state:-1 for state in states}
policy = {state:'None' for state in states}

fin_policy = []
while delta < 100:
    V_old = V.copy()
    for s in states:
        
        q = {str(action):0 for action in data[(data['Initial State'] == s)]["Action"]}
        
        for a in data[(data['Initial State'] == s)]["Action"].unique():
            
            act_df = data[(data['Initial State'] == s)][(data['Action'] == a)]
            act_df = act_df.reset_index()
            for index, row in act_df.iterrows():
                q[str(row["Action"])] += row['Transition Probability']*(float(row["Reward"])+gamma*V_old[row["Final State"]])
        max_q_key = max(q, key=q.get)
        optimum_action = max_q_key
        V[s] = q[max_q_key]
        policy[s] = [optimum_action, q[max_q_key]]
    delta = delta + 1 
    
for key, value in policy.items():
    func = (data[(data['Initial State'] == key) & (data['Action'] == value[0])]["function Name"].iloc[0])
    fin_policy.append([key,value[0], func])
    


In [6]:
df = pd.DataFrame(fin_policy, columns=['State', 'Action', 'Func'])

output = "ES_Distill_ind_Policy_mstar_3_fidThresh"+str(fid_thresh) + "alpha_0_01.xlsx"
df.to_excel(output)


In [ ]:
while fid_thresh <= 1:
    data = pd.read_excel('Final_Dis2_sep_ES_fid_threshFinal_'+str(fid_thresh)+'both_queues_transition_mStar_3queue_length=3_q_1_plink_0.7alpha_0.23104906018664842.xlsx')
    
    # Initialize the parameters
    gamma = 0.9
    delta = 0
    
    # Initialize the state space
    states = data['Initial State'].unique()
    
    
    # Initialize the action space
    actions = data['Action'].unique()
    
    V = {state:0 for state in states}
    V_old = {state:-1 for state in states}
    policy = {state:'None' for state in states}
    
    fin_policy = []
    while delta < 100:
        V_old = V.copy()
        for s in states:
            
            q = {str(action):0 for action in data[(data['Initial State'] == s)]["Action"]}
            
            for a in data[(data['Initial State'] == s)]["Action"].unique():
                
                act_df = data[(data['Initial State'] == s)][(data['Action'] == a)]
                act_df = act_df.reset_index()
                for index, row in act_df.iterrows():
                    q[str(row["Action"])] += row['Transition Probability']*(float(row["Reward"])+gamma*V_old[row["Final State"]])
            max_q_key = max(q, key=q.get)
            optimum_action = max_q_key
            V[s] = q[max_q_key]
            policy[s] = [optimum_action, q[max_q_key]]
        delta = delta + 1 
        
    for key, value in policy.items():
        func = (data[(data['Initial State'] == key) & (data['Action'] == value[0])]["function Name"].iloc[0])
        fin_policy.append([key,value[0], func])
        
    df = pd.DataFrame(fin_policy, columns=['State', 'Action', 'Func'])
    
    output = "ES_Distill_ind_Policy_mstar_3_fidThresh"+str(fid_thresh) + ".xlsx"
    df.to_excel(output)
    fid_thresh += 0.01

In [ ]:
print("a")